# Setup

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables
import requests
from package.database.user.db import UserDB
from package.database.session.db import SessionDB
from package.database.session.model import SessionEnrich, EnrichSection, EnrichQuestion, Enrich, SessionPublish, Knowledge, KnowledgeSection, KnowledgeQuestion, SessionKnowledge, SessionStep
from package.database.web_register.db import WebDB
from package.database.web_register.model import UrlRecords, UrlRecord, WebStatus
userDB = UserDB(db_name=os.getenv("DB_NAME"))
sessionDB = SessionDB(db_name=os.getenv("DB_NAME"))
webDB = WebDB(db_name=os.getenv("DB_NAME"))

BASE_URL = os.getenv("ENDPOINT")

In [31]:
userDB.drop_table()
sessionDB.drop_table()
webDB.drop_table()

,Success


In [11]:
# userDB = UserDB(db_name=os.getenv("DB_NAME"))
# sessionDB = SessionDB(db_name=os.getenv("DB_NAME"))
# webDB = WebDB(db_name=os.getenv("DB_NAME"))

In [12]:
from package.database.utils import get_UserDB, get_SessionDB

In [13]:
next(get_UserDB()).read_all()

,user_id,username,password,created_at,updated_at


In [14]:
next(get_SessionDB()).read_all()

,session_id,user_id,tone_of_voice,outline,parsed_outline,step,step_remark,knowledge,enrich,publish,created_at,updated_at


# User

## Register

In [32]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/register", headers=headers, json=payload)
response.json()

{'user_id': 'b64150c8-bef1-4275-b149-1e499d8a3311', 'username': 'bank'}

In [34]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/register", headers=headers, json=payload)
response.json()

{'detail': 'Constraint Error: Duplicate key "username: bank" violates unique constraint.'}

## Login

In [35]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/login", headers=headers, json=payload)
response.json()

{'user_id': 'b64150c8-bef1-4275-b149-1e499d8a3311', 'username': 'bank'}

In [36]:
response.status_code

200

In [37]:
user_id = response.json()['user_id']
user_id

'b64150c8-bef1-4275-b149-1e499d8a3311'

In [38]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bonk",
    "password": "P@ssw0rd",
}

response = requests.post(f"{BASE_URL}/v1/user/login", headers=headers, json=payload)
response.status_code, response.json()

(404, {'detail': "Username 'bonk' not found"})

In [39]:
headers = {"Content-type": "application/json"}
payload = {
    "username": "bank",
    "password": "P@ssward",
}

response = requests.post(f"{BASE_URL}/v1/user/login", headers=headers, json=payload)
response.status_code, response.json()

(401, {'detail': 'Incorrect password'})

# Session

## create

In [39]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": user_id,
}

response = requests.post(f"{BASE_URL}/v1/session/create", headers=headers, json=payload)
response.json()

{'response': 'success'}

## list

In [22]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": user_id,
}

response = requests.post(f"{BASE_URL}/v1/session/list", headers=headers, json=payload)
response.json()

{'response': []}

In [41]:
session_id = response.json()['response'][0]['session_id']
session_id

'3176fa54-31ed-4521-84da-2d410263972a'

In [42]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": "fake",
}

response = requests.post(f"{BASE_URL}/v1/session/list", headers=headers, json=payload)
response.json()

{'response': []}

## update-outline

In [43]:
tone_of_voice = """
Write an article as the introduction to Insurance Fraud. Make the language easy and simple for the beginners. The article aims to educate a new fraud investigator.
""".strip()

outline = """
## Overview of Insurance Fraud  
- what is insurance fraud look like  
""".strip()

# outline = """
# ## Overview of Insurance Fraud  
# - what is insurance fraud look like  
# - what is the impact on insurance fraud in USA  
# - how many types of insurance frauds are  
# - how to detect insurance fraud  
# """.strip()

headers = {"Content-type": "application/json"}
payload = {
    "session_id": session_id,
    "tone_of_voice": tone_of_voice,
    "outline": outline
}

response = requests.put(f"{BASE_URL}/v1/session/update-outline", headers=headers, json=payload)
response.json()

{'response': [{'section': 'Overview of Insurance Fraud',
   'questions': ['what is insurance fraud look like']}]}

In [44]:
headers = {"Content-type": "application/json"}
payload = {
    "user_id": user_id,
}

response = requests.post(f"{BASE_URL}/v1/session/list", headers=headers, json=payload)
response.json()

{'response': [{'session_id': '3176fa54-31ed-4521-84da-2d410263972a',
   'user_id': '5f64716a-0d76-40d6-aa94-132aad1a5ef9',
   'tone_of_voice': 'Write an article as the introduction to Insurance Fraud. Make the language easy and simple for the beginners. The article aims to educate a new fraud investigator.',
   'outline': '## Overview of Insurance Fraud  \n- what is insurance fraud look like',
   'parsed_outline': '[{"section":"Overview of Insurance Fraud","questions":["what is insurance fraud look like"]}]',
   'step': 'not start',
   'step_remark': None,
   'knowledge': None,
   'enrich': None,
   'publish': None,
   'created_at': '2025-05-05T01:54:01.973544',
   'updated_at': '2025-05-05T01:54:16.861038'}]}

In [45]:
sessionDB.read_all()

,session_id,user_id,tone_of_voice,outline,parsed_outline,step,step_remark,knowledge,enrich,publish,created_at,updated_at
0,3176fa54-31ed-4521-84da-2d410263972a,5f64716a-0d76-40d6-aa94-132aad1a5ef9,Write an article as the introduction to Insura...,## Overview of Insurance Fraud \n- what is in...,"[{""section"":""Overview of Insurance Fraud"",""que...",not start,None,None,None,None,2025-05-05 01:54:01.973544,2025-05-05 01:54:16.861038


# research

In [46]:
headers = {"Content-type": "application/json"}
payload = {
    "session_id": session_id,
    "user_id": user_id
}

response = requests.post(f"{BASE_URL}/v1/session/research", headers=headers, json=payload)
response.json()

{'response': 'success'}

In [48]:
sessionDB.get_session(session_id)

,session_id,user_id,tone_of_voice,outline,parsed_outline,step,step_remark,knowledge,enrich,publish,created_at,updated_at
0,3176fa54-31ed-4521-84da-2d410263972a,5f64716a-0d76-40d6-aa94-132aad1a5ef9,Write an article as the introduction to Insura...,## Overview of Insurance Fraud \n- what is in...,"[{""section"":""Overview of Insurance Fraud"",""que...",publish,None,"{""sections"":[{""section"":""Overview of Insurance...","{""sections"":[{""section"":""Overview of Insurance...",## Overview of Insurance Fraud\n\nAs a new fra...,2025-05-05 01:54:01.973544,2025-05-05 01:54:58.530140


In [47]:
print(sessionDB.get_session(session_id)['publish'][0])

## Overview of Insurance Fraud

As a new fraud investigator, it's essential to understand the basics of insurance fraud. Insurance fraud refers to a deceptive act against an insurance company by a person, usually for financial gain [1]. This type of fraud can be committed by anyone, including applicants, policyholders, and third-party claimants, and can result in significant financial losses for insurance companies.

So, what does insurance fraud look like? It can take many forms, but there are two main types: hard fraud and soft fraud [1]. Hard fraud is pre-meditated and planned, where the person performing the fraud deliberately causes an act that results in a fraudulent insurance claim [1]. For example, someone might stage a car accident or fake a burglary to collect insurance money. Soft fraud, on the other hand, is usually unplanned and occurs when a policyholder exaggerates a claim for a larger payout [1]. This can be harder to detect, as it often stems from a legitimate claim.



## research/knowledge

In [55]:
headers = {"Content-type": "application/json"}
payload = {
    "session_id": session_id,
    "user_id": user_id
}

response = requests.post(f"{BASE_URL}/v1/session/research/knowledge", headers=headers, json=payload)
response.json()['response']

{'sections': [{'section': 'Overview of Insurance Fraud',
   'questions': [{'question': 'what is insurance fraud look like',
     'retrieved_ids': ['7635593a-f400-4182-91f4-8b0fc69363bf',
      'f15c7809-3bca-448b-9845-9e80b219ea69',
      '0b38ed4d-2354-437d-9159-ff6018ef9a00',
      'd7cb0d21-c1fc-49ea-8e5a-bb5d1f9ee365',
      'd67794fb-6b20-4ee5-aee5-b7a53b19c7b6']}]}]}

## research/enrich

In [57]:
headers = {"Content-type": "application/json"}
payload = {
    "session_id": session_id,
    "user_id": user_id
}

response = requests.post(f"{BASE_URL}/v1/session/research/enrich", headers=headers, json=payload)
response.json()['response']

{'sections': [{'section': 'Overview of Insurance Fraud',
   'questions': [{'question': 'what is insurance fraud look like',
     'answer': "Insurance fraud refers to a deceptive act against an insurance company by a person, usually for financial gain [1]. According to the FBI, non-healthcare insurance fraud costs around $40 billion a year for insurance companies, resulting in a $400-$700 yearly increase in premiums for the average American family [1]. Insurance fraud can be committed by applicants, policyholders, and third-party claimants during a transaction to obtain benefits to which they're not entitled [1].\n\nThere are two main types of insurance fraud: hard fraud and soft fraud [1]. Hard fraud is pre-meditated and planned, where the person performing the fraud deliberately causes an act that results in a fraudulent insurance claim [1]. Soft fraud, on the other hand, is usually unplanned and occurs when a policyholder exaggerates a claim for a larger payout [1]. Soft fraud is har

## research/publish

In [59]:
headers = {"Content-type": "application/json"}
payload = {
    "session_id": session_id,
    "user_id": user_id
}

response = requests.post(f"{BASE_URL}/v1/session/research/publish", headers=headers, json=payload)
print(response.json()['response'])

## Overview of Insurance Fraud

As a new fraud investigator, it's essential to understand the basics of insurance fraud. Insurance fraud refers to a deceptive act against an insurance company by a person, usually for financial gain [1]. This type of fraud can be committed by anyone, including applicants, policyholders, and third-party claimants, and can result in significant financial losses for insurance companies.

So, what does insurance fraud look like? It can take many forms, but there are two main types: hard fraud and soft fraud [1]. Hard fraud is pre-meditated and planned, where the person performing the fraud deliberately causes an act that results in a fraudulent insurance claim [1]. For example, someone might stage a car accident or fake a burglary to collect insurance money. Soft fraud, on the other hand, is usually unplanned and occurs when a policyholder exaggerates a claim for a larger payout [1]. This can be harder to detect, as it often stems from a legitimate claim.

